# Understanding Dependencies


When an object depends on another object, if one object changes, the other might be forced to change.

Sample code follows:

In [ ]:
class Gear:
  def __init__(self, chainring, cog, rim, tire):
    self.chainring = chainring
    self.cog = cog
    self.rim = rim
    self.tire = tire

  def ratio(self):
    return self.chainring / float(self.cog)
    
  def gear_inches(self):
    return self.ratio() * Wheel(self.rim, self.tire).diameter()

class Wheel:
  def __init__(self, rim, tire):
    self.rim = rim
    self.tire = tire

  def diameter(self):
    return self.rim + (self.tire * 2)

Gear(52, 11, 26, 1.5).gear_inches()

137.0909090909091

Gear class has dependencies on Wheel class at least 4.

* Gear expects a class named Wheel to exist.
* Gear expects a Wheel instance to respond to diameter.
* Gear knows that Wheel() requires rim and tire.
* Gear knows the first argument to Wheel() must be rim, the second, tire.

These dependencies cause that Gear class will be forced to change because of a change to Wheel class.

The more Gear class knows about Wheel class, the more tightly coupled they are. The more tightly coupled two objects are, they become like a single object.

If that happens, you have to change Gear class when Wheel class change. Wheel class comes along when you want to use just Gear class. You have to test Wheel class when you want to test just Gear class.

# Writing Loosely Coupled Code


In next sample code, Gear class expects to be initialized with an object which can respond to diameter, instead of expecting a class named Wheel to exist.

In [ ]:
class Gear:
  def __init__(self, chainring, cog, wheel):
    self.chainring = chainring
    self.cog = cog
    self.wheel = wheel

  def ratio(self):
    return self.chainring / float(self.cog)

  def gear_inches(self):
    return self.ratio() * self.wheel.diameter()

class Wheel:
  def __init__(self, rim, tire):
    self.rim = rim
    self.tire = tire

  def diameter(self):
    return self.rim + (self.tire * 2)
    
Gear(52, 11, Wheel(26, 1.5)).gear_inches()

137.0909090909091

This change is very small but this change bring huge benefits. Gear can now collaborate with any object which has diameter, and object doesn’t need to be named Wheel. Through the number of code lines decrease rather than increasing, bring benefits.

## Isolate Dependency
However, this sample is ideal situation. You might be under severe constraint when you are working with existing application.



### Isolate Instance Creation



You can isolate instance creation like 2 samples as below:

Sample1 — Instance creation is moved to constructor.

In [ ]:
class Gear:
  def __init__(self, chainring, cog, rim, tire):
    self.chainring = chainring
    self.cog = cog
    self.wheel = Wheel(rim, tire)
    
  def gear_inches(self):
    return self.ratio() * self.wheel.diameter()

Sample2 — Instance creation is defined in method.

In [ ]:
class Gear:
  def __init__(self, chainring, cog, rim, tire):
    self.chainring = chainring
    self.cog = cog
    self.rim = rim
    self.tire = tire

  def gear_inches(self):
    return self.ratio() * self.wheel().diameter()
    
  def wheel(self):
    return Wheel(self.rim, self.tire)

Of course, both samples still knows too much, however, the number of dependencies in gear_inches has reduced.



### Isolate External Message



Now it’s time to turn your attention to external message, which is “sent to someone other than self”. For example, gear_inches sends ratio and wheel to self, but diameter to wheel.

To remove external dependency, you can encapsulate it in a method like this:

In [ ]:
class Gear:
  def __init__(self, chainring, cog, rim, tire):
    self.chainring = chainring
    self.cog = cog
    self.rim = rim
    self.tire = tire

  def gear_inches(self):
    return self.ratio() * self.diameter()

  def wheel(self):
    return Wheel(self.rim, self.tire)
    
  def diameter(self):
    self.wheel().diameter()

In the original code, gear_inches knows that wheel has a diameter. Gear now isolates wheel().diameter() in a separate method and gear_inches can depend on a message sent to self.


## Remove Argument-Order Dependencies


You cannot avoid having knowledge of arguments to send message as a sender. Many method requires not only arguments but also being passed in the collect order.

In this example, Gear class requires three arguments, chainring, cog and wheel, to initialize. It provides no defaults and arguments must be passed in the collect order.



### Use Dictionary for Initialization Arguments



You can use dictionary (like hash in Ruby) to avoid depending on fixed-order arguments. The next example shows this technique.



In [ ]:
class Gear:
  def __init__(self, args):
    self.chainring = args["chainring"]
    self.cog = args["cog"]
    self.wheel = args["wheel"]

  def ratio(self):
    return self.chainring / float(self.cog)

  def gear_inches(self):
    return self.ratio() * self.wheel.diameter()

class Wheel:
  def __init__(self, rim, tire):
    self.rim = rim
    self.tire = tire
    
  def diameter(self):
    return self.rim + (self.tire * 2)

Gear(
  {
    "chainring": 52
    , "cog": 11
    , "wheel": Wheel(26, 1.5)
  }
).gear_inches()

137.0909090909091

This technique has several advantages:

* Removes every dependency on argument order
* Gear is now free to add or remove initialization arguments
* No change will have side effects in other code

### Explicitly Define Defaults



You can set default of dictionary using “get” in Python as below:



In [ ]:
class Gear:
  def __init__(self, args):
    self.chainring = args.get("chainring", 52)
    self.cog = args.get("cog", 11)
    self.wheel = args["wheel"]

  def ratio(self):
    return self.chainring / float(self.cog)

  def gear_inches(self):
    return self.ratio() * self.wheel.diameter()

class Wheel:
  def __init__(self, rim, tire):
    self.rim = rim
    self.tire = tire
    
  def diameter(self):
    return self.rim + (self.tire * 2)

Gear(
  {
    "wheel": Wheel(26, 1.5)
  }
).gear_inches()

137.0909090909091

Of course we have any other way to set default, sender doesn’t need to know arguments is required using default anyway.

